In [ ]:
-- select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
-- where src = 'scale'
-- and data_source in ('scale_hpi', 'scale_michelin', 'scale_fas', 'scale_bose')
-- and inc_day in ('20211124', '20211117', '20211110', '20211103', 
-- '20211027', '20211020', '20211013','20211006')
-- and usage_flag = '1'

In [19]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import re
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

print(os.listdir('data_down'))

['inv_7.csv']


In [70]:
%%time
df = pd.read_csv('./data_down/inv_7.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)
time_cols = pd.Series(df.columns)[pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))
df9 = df

Wall time: 6.94 s


In [105]:
def load_data(data_source):
    """
    load bose data;
    所有类型的qty都要加起来哦
    只选择有多个收货日期的货物
    """
    df = df9
    df = df[df['data_source'] == data_source]
    df ['recived_date'] = pd.to_datetime(df['recived_date'])
    df = df[['wms_company_name', 'wms_warehouse_id','sku_code', 'sku_name', 'sku_desc', 'location_zone',\
        'lock_codes', 'on_hand_qty', 'in_transit_qty','allocated_qty', 'shelf_days', 
        'recived_date','usage_flag', 'inc_day']]
    # qty = pd.Series(df.columns).str.extract('(.+qty)').dropna(axis = 0)
    # df['qty'] = df[qty[0]].sum(axis = 1).round(2)
    df['qty'] = df['on_hand_qty']
    
    # bose_inv.drop_duplicates(subset = ['sku_code', 'recived_date', 'lock_codes', 'inc_day', 'qty'])
    # 没有重复的 目前看....aaa
    df = df.groupby(['recived_date', 'sku_code', 'lock_codes','inc_day', 'wms_warehouse_id'], dropna = False).agg(
    {
        'qty':sum,
        'location_zone': set
    }
    ).sort_values(['sku_code', 'recived_date']).reset_index()
    # 只选择有多个收货日期的货物
    filter0 = df.groupby(['sku_code'])['recived_date'].agg(
    {
        set
    }
        ).reset_index()

    filter0 = pd.DataFrame(filter0[filter0['set'].apply(len)> 1]['sku_code'].drop_duplicates())
    bose_inv = filter0.merge(df, on = ['sku_code'], how = 'inner')\
        .sort_values(['recived_date','sku_code', 'inc_day'])
    
    return bose_inv

bose_inv = load_data('scale_bose')

In [106]:
# # 只选择多次出库的货物
# filter = bose.groupby(['recived_date', 'sku_code'])['inc_day'].agg({set}).reset_index()
# filter = filter[filter['set'].apply(len)> 1][['recived_date','sku_code']].drop_duplicates();filter
# bose_inv.head(4)

In [107]:
%%time
df0 = pd.DataFrame()
def snapshot():
    """
    pivot table. inc_day 快照 作为 cols
    添加标记.
    """
    global df0, bose_inv
    for i in tqdm(bose_inv['sku_code'].unique(), ascii= False, colour = 'green'):
        df_out = bose_inv[bose_inv['sku_code'] == i]\
            .pivot_table(columns=['inc_day'], index = 'recived_date', values=['qty']).reset_index()
        df_out['sku_code'] = i
        df0 = pd.concat([df0, df_out], axis = 0)
    try:
        df0.columns = df0.columns.get_level_values(level=1)
    except:
        pass
    df0.columns = list(df0.columns[0:8]) + ['received_date','sku']
    df0 = df0.sort_values(['sku', 'received_date'])
    df0['mark'] = 0
    # 这个吊东`西不知道为什么是反选的 
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:7].isna().all(axis = 1) == False, 'new')
    df0['mark'] = df0['mark'].where(~df0.iloc[:,7].isna() , 'clear')
    # fill na~
    df0 = df0.fillna(0)
    bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
        .reset_index(drop = True).drop(['inc_day', 'qty'], axis = 1)
    bose_inv = bose_inv.drop_duplicates(subset = ['sku', 'received_date', 'lock_codes'])
    df0 = df0.merge(bose_inv, on = ['sku','received_date'], how = 'left')

    # may lock
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
snapshot()

100%|██████████| 1034/1034 [00:21<00:00, 47.18it/s]


Wall time: 22.4 s


In [109]:
def err_part():
    """
    findout who are the naught peach.
    err 中干掉了 new 干掉了maylock 
    """
    df_err = df0[df0['mark'] != 'new']
    # 补充可能被锁的标记
    # df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    # 干掉了maylock
    df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
    df_err['change'] = df_err.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = df_err.groupby('sku').shift(1) 
    # q = []
    # for i in shift['mark']:
    #     if pd.isna(i):
    #         q.append('clear')
    #     elif i =='clear':
    #         q.append('clear')
    #     else:
    #         q.append('check')
    # shift['mark'] = q
    # shift['mark2'] = shift.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = shift[['mark','change']]
    shift.columns = ['lag_mark', 'lag_change']
    shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
    df_err = pd.concat([df_err, shift], axis = 1)
    # df_wrong = df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] != 0)]
    # df_good  = df_err[(df_err['lag_mark'] != 'clear') | (df_err['change'] != 0)]
    return df_err
df_err = err_part()

In [110]:
# df_err.to_pickle('./data_up/df_err_bose.p')
# df0.to_pickle('./data_up/df0_bose.p')
df_err_bose = pd.read_pickle('./data_up/df_err_bose.p')
df0_bose = pd.read_pickle('./data_up/df0_bose.p')
df0_bose = df0_bose[df0_bose['wms_warehouse_id'] == 'BOSE_SH']
df_err_bose = df_err_bose[df_err_bose['wms_warehouse_id'] == 'BOSE_SH']

In [111]:

def output(df_err, df0):
    dishes = list(df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] < 0)]['sku'].unique())
    return df0[df0['sku'].isin(dishes)]
def check(sku, df0):
    return df0[(df0['lock_codes'] == '1000') & (df0['sku'].isin(sku))].sort_values(['sku','received_date'])


In [112]:
view = output(df_err_bose,df0_bose)
# 1000全新 3000退货 4000破损产品-
def ou_level_lock_codes(lock_code_to_eliminate):
    """
    正则. lock_code 需要被排除的
    """
    # output(df_err_bose,df0_bose)['sku'].unique()
    select_none_lock  = pd.DataFrame(
        view.groupby('sku')['mark'].apply(list).astype(str).str.match('.+may')
        ).reset_index()
    select_none_lock2 = pd.DataFrame(
        view.groupby('sku')['lock_codes'].apply(list).astype(str).str.match('.+'+lock_code_to_eliminate)
        ).reset_index()
    # 去重    
    bose_err_list = set(select_none_lock[~select_none_lock['mark']]['sku'].unique())
    bose_err_list2 = list(select_none_lock2[~select_none_lock2['lock_codes']]['sku'].unique())
    bose_err_list = list(bose_err_list.intersection(bose_err_list2))
    bose_err_list = list(set(bose_err_list))
    return bose_err_list

In [113]:
bose_err_list = ou_level_lock_codes('(4000|3000)')

In [117]:
bose_definite_wrong = check(bose_err_list, df0_bose)
bose_definite_wrong\
    .to_csv('./data_up/bose_fifo.csv', index = None, encoding = 'utf_8_sig')

=================================
# HP
=================================


In [115]:
%%time
# del df0 ,bose_inv
bose_inv = load_data('scale_hpi');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

100%|██████████| 1483/1483 [00:30<00:00, 48.80it/s]


Wall time: 32.6 s


In [83]:
df0_hp['lock_codes'].unique()

array(['1001-34', '1002-34', '1004-34', '1008-34', '1001-QH', '1008-QH',
       '1001-27', '1007-QH', '1007-34', '1002-27', '1002-QH', '1008-27',
       '1004-27', '1004-QH', '1006-27', '1006-34', '1007-27'],
      dtype=object)

In [118]:
# df_err.to_pickle('./data_up/df_err_hp.p')
# df0.to_pickle('./data_up/df0_hp.p')
df_err_hp = pd.read_pickle('./data_up/df_err_hp.p')
df0_hp = pd.read_pickle('./data_up/df0_hp.p')


In [119]:
output(df_err_hp, df0_hp)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
41,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-07-13,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
42,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-07-19,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
43,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-07-22,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
44,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-08-05,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
45,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-08-09,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12138,10.0,10.0,4.0,4.0,4.0,4.0,4.0,0.0,2021-09-29,Z7Y79A,clear,1001-34,HPI_WH,{L_Normal}
12188,36.0,18.0,6.0,6.0,6.0,6.0,6.0,6.0,2021-08-31,Z9M07A,0,1001-34,HPI_WH,{L_Normal}
12189,36.0,18.0,6.0,6.0,6.0,6.0,6.0,6.0,2021-08-31,Z9M07A,0,1002-34,HPI_WH,{L_Normal}
12190,36.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-09-27,Z9M07A,clear,1001-34,HPI_WH,{L_Normal}


In [120]:
view = output(df_err_hp, df0_hp)
err_list_hp = ou_level_lock_codes('(QH|27)')

In [241]:
# df0_hp[df0_hp['sku'] == '1VV21AA']

In [121]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0_hp)

# Z7Y79A 问一下
# 

In [123]:
hp_definite_wrong

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
541,496.0,496.0,238.0,0.0,0.0,0.0,0.0,0.0,2021-07-12,3WX02A,clear,1001-34,HPI_SH,{L_Normal}
542,125.0,125.0,125.0,62.0,62.0,5.0,5.0,5.0,2021-07-28,3WX02A,0,1001-34,HPI_SH,{L_Normal}
543,480.0,480.0,480.0,480.0,480.0,187.0,0.0,0.0,2021-08-26,3WX02A,clear,1001-34,HPI_SH,{L_Normal}
544,240.0,240.0,240.0,240.0,240.0,240.0,147.0,107.0,2021-09-06,3WX02A,0,1001-34,HPI_SH,{L_Normal}
545,0.0,0.0,0.0,360.0,360.0,360.0,360.0,360.0,2021-10-25,3WX02A,0,1001-34,HPI_SH,{L_Normal}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12138,10.0,10.0,4.0,4.0,4.0,4.0,4.0,0.0,2021-09-29,Z7Y79A,clear,1001-34,HPI_WH,{L_Normal}
12188,36.0,18.0,6.0,6.0,6.0,6.0,6.0,6.0,2021-08-31,Z9M07A,0,1001-34,HPI_WH,{L_Normal}
12189,36.0,18.0,6.0,6.0,6.0,6.0,6.0,6.0,2021-08-31,Z9M07A,0,1002-34,HPI_WH,{L_Normal}
12190,36.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-09-27,Z9M07A,clear,1001-34,HPI_WH,{L_Normal}


In [125]:
hp_definite_wrong[hp_definite_wrong['sku'] == 'Z7Y79A']

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
12136,8.0,4.0,6.0,6.0,6.0,6.0,6.0,2.0,2021-09-02,Z7Y79A,0,1001-34,HPI_WH,{L_Normal}
12137,8.0,4.0,6.0,6.0,6.0,6.0,6.0,2.0,2021-09-02,Z7Y79A,0,1002-34,HPI_WH,{L_Normal}
12138,10.0,10.0,4.0,4.0,4.0,4.0,4.0,0.0,2021-09-29,Z7Y79A,clear,1001-34,HPI_WH,{L_Normal}


In [126]:
hp_definite_wrong.to_csv('./data_up/hp_fifo.csv', index = None, encoding = 'utf_8_sig')

=================================
# michelin
=================================


In [65]:
df0 = []
bose_inv = []

In [127]:
%%time
del df0 ,bose_inv
bose_inv = load_data('scale_michelin');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

100%|██████████| 1360/1360 [00:30<00:00, 45.07it/s]


Wall time: 33.3 s


In [128]:
df_err.to_pickle('./data_up/df_err_mich.p')
df0.to_pickle('./data_up/df0_mich.p')
df_err_mich = pd.read_pickle('./data_up/df_err_mich.p')
df0_mich = pd.read_pickle('./data_up/df0_mich.p')


In [129]:
output(df0=df0_mich, df_err=df_err_mich)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
69,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2020-04-28,003789_101,may_lock,RETURN_IM-1IM,RT,{L_NC}
70,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2020-08-18,003789_101,clear,ONHAND_IM-1IM,RT,{nan}
71,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2020-09-18,003789_101,clear,ONHAND_IM-1IM,RT,{nan}
72,12.0,12.0,12.0,12.0,5.0,5.0,5.0,5.0,2020-10-18,003789_101,0,ONHAND_IM-1IM,RT,{nan}
73,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2020-10-21,003789_101,clear,ONHAND_IM-1IM,RT,{nan}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11492,17.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,2021-07-02,MATMICHELIN00584,0,Available,NaN,{nan}
11493,180.0,176.0,176.0,176.0,176.0,176.0,176.0,130.0,2021-08-14,MATMICHELIN00584,0,Available,NaN,{nan}
11494,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2021-10-12,MATMICHELIN00584,0,RETURN_IM-1IM,NaN,{nan}
11506,16.0,4.0,3.0,79.0,79.0,77.0,72.0,72.0,2021-07-03,MATMICHELIN00592,0,Available,NaN,{nan}


In [133]:
df0_mich['lock_codes'].str.slice(0,7).unique()

array(['Availab', 'RETURN_', 'ONHAND_', 'BLOCKED', 'INDNLIB', 'NCONHAN',
       'XXXXXXX'], dtype=object)

In [134]:
view = output(df0=df0_mich, df_err=df_err_mich)
err_list_mich = ou_level_lock_codes('(BLOCKED_TH|RETURN)')

In [135]:
def check(sku, df0):
    return df0[(df0['lock_codes'] == 'Available') & (df0['sku'].isin(sku))]

mich_definite_wrong = check(err_list_mich,df0 = df0_mich)

# MATMICHELIN00591

# 同一时间 received 货物为什么会增加
# mich dot date?

In [136]:
df0[df0['sku'] == '010432_109']

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
118,27.0,25.0,23.0,21.0,21.0,27.0,26.0,26.0,2021-08-16,010432_109,0,Available,RT,{L_label}
119,30.0,30.0,26.0,26.0,26.0,26.0,26.0,26.0,2021-09-14,010432_109,0,Available,RT,{L_label}
120,27.0,29.0,29.0,29.0,29.0,29.0,28.0,28.0,2021-10-01,010432_109,0,Available,RT,{L_label}
121,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-10-11,010432_109,clear,ONHAND_FR-16U,RT,{nan}
122,0.0,0.0,11.0,11.0,11.0,11.0,10.0,10.0,2021-10-15,010432_109,0,NCONHAND_FR-16U,RT,{nan}
123,0.0,0.0,11.0,11.0,11.0,11.0,10.0,10.0,2021-10-15,010432_109,0,ONHAND_FR-16U,RT,{nan}
124,0.0,0.0,0.0,21.0,21.0,21.0,21.0,21.0,2021-10-21,010432_109,0,Available,RT,{L_label}
125,0.0,0.0,0.0,60.0,60.0,60.0,60.0,60.0,2021-10-24,010432_109,0,Available,RT,{L_label}


In [137]:
mich_definite_wrong[mich_definite_wrong['sku'] == '010432_109']

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
118,27.0,25.0,23.0,21.0,21.0,27.0,26.0,26.0,2021-08-16,010432_109,0,Available,RT,{L_label}
119,30.0,30.0,26.0,26.0,26.0,26.0,26.0,26.0,2021-09-14,010432_109,0,Available,RT,{L_label}
120,27.0,29.0,29.0,29.0,29.0,29.0,28.0,28.0,2021-10-01,010432_109,0,Available,RT,{L_label}
124,0.0,0.0,0.0,21.0,21.0,21.0,21.0,21.0,2021-10-21,010432_109,0,Available,RT,{L_label}
125,0.0,0.0,0.0,60.0,60.0,60.0,60.0,60.0,2021-10-24,010432_109,0,Available,RT,{L_label}


In [138]:
mich_definite_wrong.to_csv('./data_up/mich_fifo.csv', index = None, encoding = 'utf_8_sig')